## 『本次練習內容』
#### 使用Xception backbone做 Trnasfer Learning


## 『本次練習目的』
  #### 了解如何使用Transfer Learning
  #### 了解Transfer Learning的優點，可以觀察模型收斂速度

##### 可以自行嘗試多種架構

## 1. 從 Keras 讀入特定模型架構 

### Resnet 50 架構 

In [1]:
#載入套件
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
import keras
from keras.layers import Input
 
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


#定義輸入影像大小
input_tensor = Input(shape=(32, 32, 3))
#include top 決定要不要加入 fully Connected Layer
'''Xception 架構'''

'''Resnet 50 架構'''
Resnet50_model=keras.applications.ResNet50(include_top=False, #不要導入全連接層，主要是由於輸出類別數量不同。
                                  weights='imagenet', #代表加載在 ImageNet 上預訓練的權值。
                                  input_tensor=input_tensor, #模型的輸入。
                                  pooling=None,  #代表不池化，直接輸出最後一層卷積層的輸出，該輸出是一個四維張量。
                                  classes=10) #圖片分類的類別數量。
#輸出Resnet 50架構
print(Resnet50_model.summary())


Using TensorFlow backend.
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

Instructions for updating:
Colocations handled automatically by placer.


C:\ProgramData\Anaconda3\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 16, 16, 64)   256         conv1[0][0]                      
___________________________________________________________________________________________

### Xception 架構

In [7]:
#載入套件
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
import keras
from keras.layers import Input
 
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


#定義輸入影像大小
input_tensor = Input(shape=(32, 32, 3))
#include top 決定要不要加入 fully Connected Layer
'''Xception 架構'''
#Xception 架構的預設輸入影像尺寸為(299x299)
Xception_model=keras.applications.xception.Xception(include_top=False, #不要導入全連接層，主要是由於輸出類別數量不同。
                                           weights='imagenet', #代表加載在 ImageNet 上預訓練的權值。
                                           input_tensor=input_tensor, #模型的輸入。
                                           pooling=None,  #代表不池化，直接輸出最後一層卷積層的輸出，該輸出是一個四維張量。
                                           classes=10) #圖片分類的類別數量。

#輸出Xception 架構
print(Xception_model.summary())

Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 15, 15, 32)   864         input_4[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 15, 15, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 15, 15, 32)   0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

## 2. 添加層數

(在原本架構後面再新增幾層，尤其當沒有導入Fully connected layers時，是必要自己加上 FC 做分類。)

In [3]:
#接上原本的output
x =Resnet50_model.output 
#自行新增新的層
x = GlobalAveragePooling2D()(x) #對空間數據(2D)新增平均池化層
x = Dense(output_dim=128, activation='relu')(x) #
x=Dropout(p=0.1)(x)
'''可以參考Cifar10實作章節'''

#重新定義輸出類別的數量
predictions = Dense(output_dim=10,activation='softmax')(x)
model = Model(inputs=Resnet50_model.input, outputs=predictions)
print('ResNet_Model深度：', len(model.layers))
'''
len(model.layers)看看模型的深度，以決定鎖定的層數
'''

ResNet_Model深度： 179


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=128)`
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`
  # Remove the CWD from sys.path while we load stuff.


## 3. 鎖定特定幾層不要更新權重

定義要 Freeze 的層數，之後就跟訓練一般模型一樣了

In [4]:
for layer in model.layers[:100]:
    layer.trainable = False
for layer in model.layers[100:]:
    layer.trainable = True

## 準備 Cifar 10 資料

In [5]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7)
        return X_train, X_test
    
    
## Normalize Training and Testset    
x_train, x_test = normalize(x_train, x_test) 

## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

(50000, 32, 32, 3)


## Training

In [6]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(x_train,y_train,batch_size=32,epochs=100)

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
50000/50000 [==============================] - 124s 2ms/step - loss: 1.3574 - accuracy: 0.5599
Epoch 2/100
50000/50000 [==============================] - 117s 2ms/step - loss: 0.9769 - accuracy: 0.6693
Epoch 3/100
50000/50000 [==============================] - 115s 2ms/step - loss: 0.8565 - accuracy: 0.7069
Epoch 4/100
50000/50000 [==============================] - 114s 2ms/step - loss: 0.7586 - accuracy: 0.7430
Epoch 5/100
50000/50000 [==============================] - 114s 2ms/step - loss: 0.6665 - accuracy: 0.7738
Epoch 6/100
50000/50000 [==============================] - 117s 2ms/step - loss: 0.5782 - accuracy: 0.8017
Epoch 7/100
50000/50000 [==============================] - 116s 2ms/step - loss: 0.5761 - accuracy: 0.7991
Epoch 8/100
50000/50000 [==============================] - 115s 2ms/step - loss: 0.4851 - accuracy: 0.8290
Epoch 9/100
50000/50000 [==============================] - 118s 2ms/step - loss: 0.5109 - accura

50000/50000 [==============================] - 113s 2ms/step - loss: 0.0789 - accuracy: 0.9776
Epoch 75/100
50000/50000 [==============================] - 114s 2ms/step - loss: 0.0653 - accuracy: 0.9800
Epoch 76/100
50000/50000 [==============================] - 113s 2ms/step - loss: 0.0579 - accuracy: 0.9822
Epoch 77/100
50000/50000 [==============================] - 113s 2ms/step - loss: 0.0628 - accuracy: 0.9820
Epoch 78/100
50000/50000 [==============================] - 113s 2ms/step - loss: 0.0835 - accuracy: 0.9781
Epoch 79/100
50000/50000 [==============================] - 113s 2ms/step - loss: 0.0533 - accuracy: 0.9834
Epoch 80/100
50000/50000 [==============================] - 114s 2ms/step - loss: 0.0619 - accuracy: 0.9815
Epoch 81/100
50000/50000 [==============================] - 113s 2ms/step - loss: 0.0629 - accuracy: 0.9812
Epoch 82/100
50000/50000 [==============================] - 113s 2ms/step - loss: 0.0570 - accuracy: 0.9839
Epoch 83/100
50000/50000 [===============

# 儲存模型

In [7]:
model.save('Day021_Transfer Learning_HW.h5')